# Phi-4 Mini ONNX ಸಮಾನಾಂತರ ಫಂಕ್ಷನ್ ಕರೆ ಮಾರ್ಗದರ್ಶನ

This notebook demonstrates how to use Phi-4 Mini with ONNX Runtime GenAI for parallel function calling. Function calling allows the model to intelligently invoke external tools and APIs based on user requests.

## ಅವಲೋಕನ

In this tutorial, you'll learn how to:
- Phi-4 Mini ಅನ್ನು ONNX Runtime GenAI ರೊಂದಿಗೆ ಸೆಟ್ ಅಪ್ ಮಾಡುವುದು
- ವಿಮಾನಗಳು ಮತ್ತು ಹೋಟೆಲ್ಗಳನ್ನು ಬುಕಿಂಗ್ ಮಾಡಲು ಫಂಕ್ಷನ್ ಸ್ಕೀಮಾಗಳನ್ನು ವ್ಯಾಖ್ಯಾನಿಸುವುದು
- ರಚನಾತ್ಮಕ ಔಟ್ಪುಟ್‌ಗೆ Lark ವ್ಯಾಕರಣದೊಂದಿಗೆ ಮಾರ್ಗದರ್ಶಿತ ಜನರೇಷನ್ ಅನ್ನು ಬಳಸು
- ಸಂಕೀರ್ಣ ಪ್ರಯಾಣ ಬುಕಿಂಗ್ ಸಂದರ್ಭಗಳಿಗೆ ಸಮಾನಾಂತರ ಫಂಕ್ಷನ್ ಕರೆಗಳನ್ನು ನಿರ್ವಹಿಸುವುದು

## ಪೂರ್ವಾವಶ್ಯಕತೆಗಳು

Before running this notebook, ensure you have:
- Phi-4 Mini ONNX ಮಾದರಿಯನ್ನು ಡೌನ್ಲೋಡ್ ಮಾಡಿರುವುದು
- `onnxruntime-genai` ಪ್ಯಾಕೇಜ್ ಅನ್ನು ಇನ್‌ಸ್ಟಾಲ್ ಮಾಡಿರುವುದು
- ಫಂಕ್ಷನ್ ಕಾಲಿಂಗ್ ಸಂकल्पನೆಗಳ ಕುರಿತು ಮೂಲಭೂತ ಅರಿವು


## ಹಂತ 1: ಅಗತ್ಯ ಲೈಬ್ರರಿಗಳನ್ನು ಆಮದು ಮಾಡಿ

ಮೊದಲು, ನಮ್ಮ ಫಂಕ್ಷನ್ ಕರೆ ಅನುಷ್ಠಾನಕ್ಕಾಗಿ ಅಗತ್ಯವಿರುವ ಲೈಬ್ರರಿಗಳನ್ನು ಆಮದು ಮಾಡಿಕೊಳ್ಳುತ್ತೇವೆ.


In [1]:
import json

In [2]:
import onnxruntime_genai as og

## ಹಂತ 2: ಮಾದರಿ ಸ್ಥಾಪನೆ ಮತ್ತು ಸಂರಚನೆ

ಈಗ ನಾವು Phi-4 Mini ONNX ಮಾದರಿಯನ್ನು ಸಂರಚಿಸುತ್ತೇವೆ. ದಯವಿಟ್ಟು ಮಾರ್ಗವನ್ನು ನಿಮ್ಮ ನಿಜವಾದ ಮಾದರಿ ಡೈರೆಕ್ಟರಿಯಿಂದ ಬದಲಾಯಿಸುವುದು ಖಚಿತಪಡಿಸಿಕೊಳ್ಳಿ.


In [ ]:
# TODO: Replace with your actual Phi-4 Mini ONNX model path
# Download from: https://huggingface.co/microsoft/Phi-4-mini-onnx
path = 'Your phi-4-mini-onnx path'  # Update this path!

In [4]:
config = og.Config(path)

In [5]:
model = og.Model(config)

In [6]:
tokenizer = og.Tokenizer(model)
tokenizer_stream = tokenizer.create_stream()

## ಹಂತ 3: ಉತ್ಪಾದನಾ ಪರಾಮೀಟರ್‌ಗಳನ್ನು ಸಂರಚಿಸಿ

ಮಾದರಿಯ ಔಟ್‌ಪುಟ್ ವರ್ತನೆಯನ್ನು ನಿಯಂತ್ರಿಸಲು ಉತ್ಪಾದನಾ ಪರಾಮೀಟರ್‌ಗಳನ್ನು ಹೊಂದಿಸಿ. ಈ ಸೆಟ್ಟಿಂಗ್‌ಗಳು ಫಂಕ್ಷನ್ ಕರೆಗಾಗಿ ನಿರ್ಧಾರಾತ್ಮಕ ಮತ್ತು ಕೇಂದ್ರೀಕೃತ ಪ್ರತಿಕ್ರಿಯೆಗಳನ್ನು ಖಚಿತಪಡಿಸುತ್ತವೆ.


In [7]:
# Configure generation parameters for deterministic function calling
search_options = {}
search_options['max_length'] = 4096      # Maximum tokens to generate
search_options['temperature'] = 0.00001  # Very low temperature for deterministic output
search_options['top_p'] = 1.0            # Nucleus sampling parameter
search_options['do_sample'] = False       # Disable sampling for consistent results

## ಹಂತ 4: ಲಭ್ಯವಿರುವ ಕಾರ್ಯಗಳನ್ನು ನಿರ್ಧರಿಸಿ

ಇಲ್ಲಿ ನಾವು ನಮ್ಮ AI ಸಹಾಯಕ ಕರೆ ಮಾಡಬಹುದಾದ ಕಾರ್ಯಗಳನ್ನು ನಿರ್ಧರಿಸುತ್ತೇವೆ. ಈ ಉದಾಹರಣೆಯಲ್ಲಿ, ನಮಗೆ ಪ್ರಯಾಣಕ್ಕೆ ಸಂಬಂಧಿಸಿದ ಎರಡು ಕಾರ್ಯಗಳಿವೆ:
1. **booking_flight_tickets**: ವಿಮಾನ ನಿಲ್ದಾಣಗಳ ನಡುವೆ ವಿಮಾನ ಟಿಕೆಟ್‌ಗಳನ್ನು ಬುಕ್ ಮಾಡಲು
2. **booking_hotels**: ಹೋಟೆಲ್ ವಾಸಸ್ಥಳಗಳನ್ನು ಬುಕ್ ಮಾಡಲು

ಈ ಕಾರ್ಯ ವ್ಯಾಖ್ಯಾನಗಳು OpenAI ರ function calling schema ಸ್ವರೂಪವನ್ನು ಅನುಸರಿಸುತ್ತವೆ.


In [8]:
tool_list = '[{"name": "booking_flight_tickets", "description": "booking flights", "parameters": {"origin_airport_code": {"description": "The name of Departure airport code", "type": "string"}, "destination_airport_code": {"description": "The name of Destination airport code", "type": "string"}, "departure_date": {"description": "The date of outbound flight", "type": "string"}, "return_date": {"description": "The date of return flight", "type": "string"}}}, {"name": "booking_hotels", "description": "booking hotel", "parameters": {"destination": {"description": "The name of the city", "type": "string"}, "check_in_date": {"description": "The date of check in", "type": "string"}, "checkout_date": {"description": "The date of check out", "type": "string"}}}]'

## ಹಂತ 5: ವ್ಯಾಕರಣ ರಚನೆಗಾಗಿ ಸಹಾಯಕ ಕಾರ್ಯಗಳು

ಈ ಸಹಾಯಕ ಕಾರ್ಯಗಳು ನಮ್ಮ ಕಾರ್ಯ ನಿರ್ಣಯಗಳನ್ನು Lark ವ್ಯಾಕರಣ ಸ್ವರೂಪಕ್ಕೆ ಪರಿವರ್ತಿಸುತ್ತವೆ, ಇದು ONNX Runtime GenAI ಮೂಲಕ ಮಾರ್ಗದರ್ಶಿತ ರಚನೆಗಾಗಿ ಬಳಸಲಾಗುತ್ತದೆ. ಇದರಿಂದ ಮಾದರಿ ಸರಿಯಾದ JSON ಸ್ವರೂಪದಲ್ಲಿ ಮಾನ್ಯ ಕಾರ್ಯ ಕರೆಗಳನ್ನು ನಿರ್ಗತಿಸುವುದು ಖಚಿತವಾಗುತ್ತದೆ.


In [9]:
def get_lark_grammar(input_tools):
    tools_list = get_tools_list(input_tools)
    prompt_tool_input = create_prompt_tool_input(tools_list)
    if len(tools_list) == 1:
        # output = ("start: TEXT | fun_call\n" "TEXT: /[^{](.|\\n)*/\n" " fun_call: <|tool_call|> %json " + json.dumps(tools_list[0]))
        output = ("start: TEXT | fun_call\n" "TEXT: /[^{](.|\\n)*/\n" " fun_call: <|tool_call|> %json " + json.dumps(convert_tool_to_grammar_input(tools_list[0])))
        return prompt_tool_input, output
    else:
        return prompt_tool_input, "start: TEXT | fun_call \n TEXT: /[^{](.|\n)*/ \n fun_call: <|tool_call|> %json {\"anyOf\": [" + ','.join([json.dumps(tool) for tool in tools_list]) + "]}"


In [10]:
def get_tools_list(input_tools):
    # input_tools format: '[{"name": "fn1", "description": "fn details", "parameters": {"p1": {"description": "details", "type": "string"}}},
    # {"fn2": 2},{"fn3": 3}]'
    tools_list = []
    try:
        tools_list = json.loads(input_tools)
    except json.JSONDecodeError:
        raise ValueError("Invalid JSON format for tools list, expected format: '[{\"name\": \"fn1\"},{\"name\": \"fn2\"}]'")
    if len(tools_list) == 0:
        raise ValueError("Tools list cannot be empty")
    return tools_list

In [11]:
def create_prompt_tool_input(tools_list):
    tool_input = str(tools_list[0])
    for tool in tools_list[1:]:
        tool_input += ',' + str(tool)
    return tool_input

In [12]:
def convert_tool_to_grammar_input(tool):
    param_props = {}
    required_params = []
    for param_name, param_info in tool.get("parameters", {}).items():
        param_props[param_name] = {
            "type": param_info.get("type", "string"),
            "description": param_info.get("description", "")
        }
        required_params.append(param_name)
    output_schema = {
        "description": tool.get('description', ''),
        "type": "object",
        "required": ["name", "parameters"],
        "additionalProperties": False,
        "properties": {
            "name": { "const": tool["name"] },
            "parameters": {
                "type": "object",
                "properties": param_props,
                "required": required_params,
                "additionalProperties": False
            }
        }
    }
    if len(param_props) == 0:
        output_schema["required"] = ["name"]
    return output_schema

In [13]:
get_lark_grammar(tool_list)

("{'name': 'booking_flight_tickets', 'description': 'booking flights', 'parameters': {'origin_airport_code': {'description': 'The name of Departure airport code', 'type': 'string'}, 'destination_airport_code': {'description': 'The name of Destination airport code', 'type': 'string'}, 'departure_date': {'description': 'The date of outbound flight', 'type': 'string'}, 'return_date': {'description': 'The date of return flight', 'type': 'string'}}},{'name': 'booking_hotels', 'description': 'booking hotel', 'parameters': {'destination': {'description': 'The name of the city', 'type': 'string'}, 'check_in_date': {'description': 'The date of check in', 'type': 'string'}, 'checkout_date': {'description': 'The date of check out', 'type': 'string'}}}",
 'start: TEXT | fun_call \n TEXT: /[^{](.|\n)*/ \n fun_call: <|tool_call|> %json {"anyOf": [{"name": "booking_flight_tickets", "description": "booking flights", "parameters": {"origin_airport_code": {"description": "The name of Departure airport c

## ಹಂತ 6: ವ್ಯಾಕರಣ ರಚನೆ ಪರೀಕ್ಷೆ

ನಮ್ಮ ವ್ಯಾಕರಣ ರಚನೆ ಕಾರ್ಯಗಳನ್ನು ಪರೀಕ್ಷಿಸಿ ಅವು ನಮ್ಮ ಸಾಧನ ವ್ಯಾಖ್ಯಾನಗಳನ್ನು ಸರಿಯಾದ ಫಾರ್ಮ್ಯಾಟ್‌ಗೆ ಹೇಗೆ ಪರಿವರ್ತಿಸುತ್ತವೆ ಎಂಬುದನ್ನು ನೋಡೋಣ.


In [14]:
prompt_tool_input, guidance_input = get_lark_grammar(tool_list)

## ಹಂತ 7: ಸಿಸ್ಟಂ ಪ್ರಾಂಪ್ಟ್ ಮತ್ತು ಜನರೇಟರ್ ಅನ್ನು ಸಿದ್ಧಪಡಿಸಿ

ಇದೀಗ ನಾವು ಮಾದರಿಗೆ ಲಭ್ಯವಿರುವ ಉಪಕರಣಗಳ ಬಗ್ಗೆ ತಿಳಿಸುವ ಸಿಸ್ಟಮ್ ಪ್ರಾಂಪ್ಟ್ ಅನ್ನು ರಚಿಸಿ ಮತ್ತು ಮಾರ್ಗದರ್ಶಿತ ಜನನೆ ಪರಾಮೀಟರ್‌ಗಳೊಂದಿಗೆ ಜನರೇಟರ್ ಅನ್ನು ಸಂರಚಿಸೋಣ


In [15]:
# Define the system prompt that introduces the assistant and its capabilities
system_prompt = "You are a helpful assistant with these tools."

In [16]:
# Format the system message with tools information
messages = f"""[{{"role": "system", "content": "{system_prompt}", "tools": "{prompt_tool_input}"}}]"""

In [17]:
# Apply chat template to format the system prompt properly
tokenizer_input_system_prompt = tokenizer.apply_chat_template(messages=messages, add_generation_prompt=False)

In [18]:
tokenizer_input_system_prompt

"<|system|>You are a helpful assistant with these tools.<|tool|>{'name': 'booking_flight_tickets', 'description': 'booking flights', 'parameters': {'origin_airport_code': {'description': 'The name of Departure airport code', 'type': 'string'}, 'destination_airport_code': {'description': 'The name of Destination airport code', 'type': 'string'}, 'departure_date': {'description': 'The date of outbound flight', 'type': 'string'}, 'return_date': {'description': 'The date of return flight', 'type': 'string'}}},{'name': 'booking_hotels', 'description': 'booking hotel', 'parameters': {'destination': {'description': 'The name of the city', 'type': 'string'}, 'check_in_date': {'description': 'The date of check in', 'type': 'string'}, 'checkout_date': {'description': 'The date of check out', 'type': 'string'}}}<|/tool|><|end|><|endoftext|>"

In [19]:
input_tokens = tokenizer.encode(tokenizer_input_system_prompt)

In [20]:
input_tokens = input_tokens[:-1]

In [21]:
system_prompt_length = len(input_tokens)

## ಹಂತ 8: ಮಾರ್ಗದರ್ಶಿತ ಜನರೇಶನ್ ಬಳಸಿಕೊಂಡು ಜನರೇಟರ್ ಅನ್ನು ಪ್ರಾರಂಭಿಸಿ

ಈಗ ನಾವು ನಮ್ಮ ಕಾನ್ಫಿಗರ್ ಮಾಡಿದ ಪರಿಮಾಣಗಳೊಂದಿಗೆ ಜನರೇಟರ್ ಅನ್ನು ರಚಿಸಿ ಮಾರ್ಗದರ್ಶಿತ ಜನರೇಶನ್‌ಗೆ Lark ವ್ಯಾಕರಣವನ್ನು ಅನ್ವಯಿಸೋಣ.


In [22]:
# Create generator parameters and apply search options
params = og.GeneratorParams(model)
params.set_search_options(**search_options)

In [23]:
# Apply Lark grammar for guided generation to ensure valid function call format
params.set_guidance("lark_grammar", guidance_input)

In [24]:
generator = og.Generator(model, params)

In [25]:
generator.append_tokens(input_tokens)

## ಹಂತ 9: ಸಮಕಾಲೀನ ಫಂಕ್ಷನ್ ಕರೆಗಳನ್ನು ಪರೀಕ್ಷಿಸಿ

ಈಗ ಹಲವಾರು ಫಂಕ್ಷನ್‌ಗಳನ್ನು ಕರೆಮಾಡಬೇಕಾಗುವ ಒಂದು ಜಟಿಲ ಪ್ರಯಾಣ ಬುಕ್ಕಿಂಗ್ ಸ್ಥಿತಿಯನ್ನು ಬಳಸಿ ನಮ್ಮ ಸೆಟ್‌ಅಪ್ ಅನ್ನು ಪರೀಕ್ಷಿಸೋಣ.


In [26]:
# Complex travel booking request that requires both flight and hotel booking
text = "book flight ticket from Beijing to Paris(using airport code) in 2025-12-04 to 2025-12-10 , then book hotel from 2025-12-04 to 2025-12-10 in Paris"

In [27]:
# Format user message and apply chat template
messages = f"""[{{"role": "user", "content": "{text}"}}]"""

# Apply Chat Template for user input
user_prompt = tokenizer.apply_chat_template(messages=messages, add_generation_prompt=True)
input_tokens = tokenizer.encode(user_prompt)
generator.append_tokens(input_tokens)

In [28]:
user_prompt

'<|user|>book flight ticket from Beijing to Paris(using airport code) in 2025-12-04 to 2025-12-10 , then book hotel from 2025-12-04 to 2025-12-10 in Paris<|end|><|assistant|>'

### ರೂಪಗೊಳಿಸಿದ ಬಳಕೆದಾರನ ಪ್ರಾಂಪ್ಟ್ ವೀಕ್ಷಿಸಿ


### ಫಂಕ್ಷನ್ ಕಾಲ್‌ಗಳನ್ನು ರಚಿಸಿ

ಮಾದರಿ ಈಗ ಬಳಕೆದಾರರ ವಿನಂತಿಯ ಆಧಾರದ ಮೇಲೆ ರಚಿತ ಫಂಕ್ಷನ್ ಕಾಲ್‌ಗಳನ್ನು ರಚಿಸಲಿದೆ. ಮಾರ್ಗದರ್ಶಿತ ತಯಾರಿಕೆಗೆ ಧನ್ಯವಾದಗಳು — ಔಟ್‌ಪುಟ್ ನೇರವಾಗಿ ಕಾರ್ಯಗತಗೊಳಿಸಬಹುದಾದ ಮಾನ್ಯ JSON ಸ್ವರೂಪದಲ್ಲಿರುತ್ತದೆ.

**ನಿರೀಕ್ಷಿತ ಔಟ್‌ಪುಟ್**: ಮಾದರಿ ಕೆಳಗಿನ ಫಂಕ್ಷನ್ ಕಾಲ್‌ಗಳನ್ನು ರಚಿಸಬೇಕು:
1. `booking_flight_tickets` ನೊಂದಿಗೆ ಬೀಜಿಂಗ್ (PEK) ರಿಂದ ಪ್ಯಾರಿಸ್ (CDG) ಗೆ ಸಂಬಂಧಿಸಿದ ವಿವರಗಳು
2. `booking_hotels` ನೊಂದಿಗೆ ಪ್ಯಾರಿಸ್ ವಸತಿ ವಿವರಗಳು

ಕೆಳಗಿನ ಸೆಲ್ ಅನ್ನು ಓಡಿಸಿ ಜೀವಂತ ಉತ್ಪಾದನೆಯನ್ನು ನೋಡಲು:


In [29]:
# Generate tokens one by one and stream the output
while not generator.is_done():
    generator.generate_next_token()
    new_token = generator.get_next_tokens()[0]
    print(tokenizer_stream.decode(new_token), end='', flush=True)

[{"name": "booking_flight_tickets", "arguments": {"origin_airport_code": "PEKK", "destination_airport_code": "CDG", "departure_date": "2025-12-04", "return_date": "2025-12-10"}}, {"name": "booking_hotels", "arguments": {"destination": "Paris", "check_in_date": "2025-12-04", "checkout_date": "2025-12-10"}}]

## ಸಮಾರೋಪ

🎉 **ಶುಭಾಶಯಗಳು!** ನೀವು ONNX Runtime GenAI ಬಳಸಿ Phi-4 Mini ಜೊತೆಗೆ ಸಮಕಾಲೀನ ಫಂಕ್ಷನ್ ಕರೆಗಳನ್ನು ಯಶಸ್ವಿಯಾಗಿ ಅನುಷ್ಠಾನಗೊಳಿಸಿದ್ದೀರಿ.

### ನೀವು ಏನು ಕಲಿತಿರಿ:

1. **ಮಾದರಿ ಸಂರಚನೆ**: ONNX Runtime GenAI ಬಳಸಿ Phi-4 Mini ಅನ್ನು ಹೇಗೆ ಸಂರಚಿಸಬೇಕು
2. **ಫಂಕ್ಷನ್ ವ್ಯಾಖ್ಯಾನ**: AI ಫಂಕ್ಷಣ್ ಕರೆಗಳಿಗೆ ಫಂಕ್ಷನ್ ಸ್ಕೀಮಾಗಳನ್ನು ಹೇಗೆ ವ್ಯಾಖ್ಯಾನಿಸಬೇಕು
3. **ಮಾರ್ಗದರ್ಶಿತ ಉತ್ಪಾದನೆ**: ಸಂರಚನಿತ ಔಟ್‌ಪುಟ್ ನಿರ್ಮಾಣಕ್ಕಾಗಿ Lark ವ್ಯಾಕರಣವನ್ನು ಹೇಗೆ ಬಳಸಬೇಕು
4. **ಸಮಕಾಲೀನ ಫಂಕ್ಷನ್ ಕರೆಗಳು**: ಅನೇಕ ಫಂಕ್ಷನ್ ಕರೆಗಳನ್ನು ಅಗತ್ಯವಿರುವ ಜಟಿಲ ವಿನಂತಿಗಳನ್ನು ಹೇಗೆ ನಿರ್ವಹಿಸಬೇಕು

### ಪ್ರಮುಖ ಲಾಭಗಳು:

- ✅ **ಸಂರಚನಿತ ಔಟ್‌ಪುಟ್**: ಮಾರ್ಗದರ್ಶಿತ ಉತ್ಪಾದನೆ ಮಾನ್ಯ JSON ಫಂಕ್ಷನ್ ಕರೆಗಳನ್ನು ಖಚಿತಪಡಿಸುತ್ತದೆ
- ✅ **ಸಮಕಾಲೀನ ಪ್ರಾಸೆಸಿಂಗ್**: ಒಂದು ವಿನಂತಿಯಲ್ಲಿ ಅನೇಕ ಫಂಕ್ಷನ್ ಕರೆಗಳನ್ನು ನಿರ್ವಹಿಸಬಹುದು
- ✅ **ಉನ್ನತ ಕಾರ್ಯಕ್ಷಮತೆ**: ONNX Runtime ಆಪ್ಟಿಮೈಸ್ ಮಾಡಿದ ಇನ್ಫರೆನ್ಸ್ ಒದಗಿಸುತ್ತದೆ
- ✅ **ಬದಲಾಯಿಸಬಹುದಾದ ಸ್ಕೀಮಾ**: ಫಂಕ್ಷನ್ ವ್ಯಾಖ್ಯಾನಗಳನ್ನು ಸೇರಿಸುವುದು ಅಥವಾ ಬದಲಾಯಿಸುವುದು ಸುಲಭ

### ಸಂಪನ್ಮೂಲಗಳು:

- [Phi-4 Mini ಡಾಕ್ಯುಮೆಂಟೇಷನ್](https://huggingface.co/microsoft/Phi-4-mini-onnx)
- [ONNX Runtime GenAI ಡಾಕ್ಯುಮೆಂಟೇಷನ್](https://onnxruntime.ai/docs/genai/)
- [ಫಂಕ್ಷನ್ ಕರೆಗಳ ಅತ್ಯುತ್ತಮ ಅಭ್ಯಾಸಗಳು](https://platform.openai.com/docs/guides/function-calling)


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
ನಿರಾಕರಣೆ:
ಈ ದಸ್ತಾವೇಜನ್ನು AI ಅನುವಾದ ಸೇವೆ Co-op Translator (https://github.com/Azure/co-op-translator) ಬಳಸಿ ಅನುವಾದಿಸಲಾಗಿದೆ. ನಾವು ಶುದ್ದತೆಗೆ ಪ್ರಯತ್ನಿಸಿದರೂ, ಸ್ವಯಂಚಾಲಿತ ಅನುವಾದಗಳಲ್ಲಿ ದೋಷಗಳು ಅಥವಾ ಅಸಮರ್ಪಕತೆಗಳು ಇದ್ದಿರಬಹುದೆಂದು ದಯವಿಟ್ಟು ಗಮನಿಸಿ. ಮೂಲ ಭಾಷೆಯಲ್ಲಿ ಇರುವ ಮೂಲ ದಸ್ತಾವೇಜೆಯನ್ನು ಅಧಿಕೃತ ಮೂಲವೆಂದು ಪರಿಗಣಿಸಬೇಕು. ಗಂಭೀರ ಮಾಹಿತಿಗಾಗಿ ವೃತ್ತಿಪರ ಮಾನವ ಅನುವಾದವನ್ನು ಶಿಫಾರಸು ಮಾಡಲಾಗುತ್ತದೆ. ಈ ಅನುವಾದದ ಬಳಕೆಯಿಂದ ಉಂಟಾಗುವ ಯಾವುದೇ ಭ್ರಮೆ ಅಥವಾ ತಪ್ಪಾಗಿ ಅರ್ಥಮಾಡಿಕೊಳ್ಳುವ ಪರಿಣಾಮಗಳಿಗೆ ನಾವು ಜವಾಬ್ದಾರರಾಗುವುದಿಲ್ಲ.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
